# Vector Poisson equation 

In this example we consider the vector Poisson equation with homogeneous Dirichlet boundary conditions:

$$
\begin{align}
  - \nabla^2 \mathbf{u} = \mathbf{f} \quad \mbox{in} ~ \Omega, \quad \quad 
  \mathbf{u} = 0            \quad \mbox{on} ~ \partial \Omega.
\end{align}
$$

## The Variational Formulation

The corresponding variational formulation, using $\mathbf{H}^1$ formulation, *i.e.* all components are in $H^1$, reads 

$$
\begin{align}
  \text{find $\mathbf{u} \in V$ such that} \quad 
  a(\mathbf{u},\mathbf{v}) = l(\mathbf{v}) \quad \forall \mathbf{v} \in V,
\end{align}
$$

where 

- $V \subset \mathbf{H}_0^1(\Omega)$, 
- $a(\mathbf{u},\mathbf{v}) := \int_{\Omega} \nabla \mathbf{u} : \nabla \mathbf{v} ~ d\Omega$,
- $l(\mathbf{v}) := \int_{\Omega} \mathbf{f} \cdot \mathbf{v} ~ d\Omega$.

## Formal Model

In [9]:
from sympde.expr import BilinearForm, LinearForm, integral
from sympde.expr     import find, EssentialBC, Norm, SemiNorm
from sympde.topology import VectorFunctionSpace, Cube, element_of
from sympde.calculus import grad, inner, dot

from sympy import pi, sin, Tuple, Matrix

domain = Cube()

V = VectorFunctionSpace('V', domain)

x,y,z = domain.coordinates

u,v = [element_of(V, name=i) for i in ['u', 'v']]

# bilinear form
a = BilinearForm((u,v), integral(domain , inner(grad(v), grad(u))))

# linear form
f1 = 3*pi**2*sin(pi*x)*sin(pi*y)*sin(pi*z)
f2 = 3*pi**2*sin(pi*x)*sin(pi*y)*sin(pi*z)
f3 = 3*pi**2*sin(pi*x)*sin(pi*y)*sin(pi*z)
f = Tuple(f1, f2, f3)

l = LinearForm(v, integral(domain, dot(f,v)))

# Dirichlet boundary conditions
bc = [EssentialBC(u, 0, domain.boundary)]

# Variational problem
equation   = find(u, forall=v, lhs=a(u, v), rhs=l(v), bc=bc)

## Discretization

We shall need the **discretize** function from **PsyDAC**.

In [10]:
from psydac.api.discretization import discretize

In [11]:
degree = [2,2,2]
ncells = [8,8,8]

In [12]:
# Create computational domain from topological domain
domain_h = discretize(domain, ncells=ncells, comm=None)

# Create discrete spline space
Vh = discretize(V, domain_h, degree=degree)

# Discretize equation
equation_h = discretize(equation, domain_h, [Vh, Vh])

## Solving the PDE

In [13]:
uh = equation_h.solve()

## Computing the error norm

When the analytical solution is available, you might be interested in computing the $L^2$ norm or $H^1_0$ semi-norm.
SymPDE allows you to do so, by creating the **Norm** object.
In this example, the analytical solution is given by

$$
u_e = \sin(\pi x) \sin(\pi y) \sin(\pi z)
$$

### Computing the $L^2$ norm

In [15]:
ue1 = sin(pi*x)*sin(pi*y)*sin(pi*z)
ue2 = sin(pi*x)*sin(pi*y)*sin(pi*z)
ue3 = sin(pi*x)*sin(pi*y)*sin(pi*z)
ue = Tuple(ue1, ue2, ue3)

u = element_of(V, name='u')

error = Matrix([u[0]-ue[0], u[1]-ue[1], u[2]-ue[2]])

# create the formal Norm object
l2norm = Norm(error, domain, kind='l2')

# discretize the norm
l2norm_h = discretize(l2norm, domain_h, Vh)

# assemble the norm
l2_error = l2norm_h.assemble(u=uh)

# print the result
print(l2_error)

0.0003267962376349343


### Computing the $H^1$ semi-norm

In [16]:
# create the formal Norm object
h1norm = SemiNorm(error, domain, kind='h1')

# discretize the norm
h1norm_h = discretize(h1norm, domain_h, Vh)

# assemble the norm
h1_error = h1norm_h.assemble(u=uh)

# print the result
print(h1_error)

0.01956220393797005


### Computing the $H^1$ norm

In [17]:
# create the formal Norm object
h1norm = Norm(error, domain, kind='h1')

# discretize the norm
h1norm_h = discretize(h1norm, domain_h, Vh)

# assemble the norm
h1_error = h1norm_h.assemble(u=uh)

# print the result
print(h1_error)

0.01956493339348906
